In [45]:
import datetime
import time
import tarfile

import boto3
import pandas as pd
import numpy as np
from sagemaker import get_execution_role
import sagemaker
from sklearn.model_selection import train_test_split
from sklearn.datasets import load_boston


sm_boto3 = boto3.client("sagemaker")

sess = sagemaker.Session()

region = sess.boto_session.region_name

bucket = sess.default_bucket()  # this could also be a hard-coded bucket name

print("Using bucket " + bucket)



Using bucket sagemaker-us-east-2-296976521096


In [46]:
# we use the Boston housing dataset
data = load_boston()

In [47]:
X_train, X_test, y_train, y_test = train_test_split(
    data.data, data.target, test_size=0.25, random_state=42
)

trainX = pd.DataFrame(X_train, columns=data.feature_names)
trainX["target"] = y_train

testX = pd.DataFrame(X_test, columns=data.feature_names)
testX["target"] = y_test



In [48]:
trainX.head()

trainX.to_csv("boston_train.csv")
testX.to_csv("boston_test.csv")

# send data to S3. SageMaker will take training data from s3
trainpath = sess.upload_data(
    path="boston_train.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)

testpath = sess.upload_data(
    path="boston_test.csv", bucket=bucket, key_prefix="sagemaker/sklearncontainer"
)



In [49]:
%%writefile script.py

import argparse
import joblib
import os

import numpy as np
import pandas as pd
from sklearn.ensemble import RandomForestRegressor


# inference functions ---------------
def model_fn(model_dir):
    clf = joblib.load(os.path.join(model_dir, "model.joblib"))
    return clf


if __name__ == "__main__":

    print("extracting arguments")
    parser = argparse.ArgumentParser()

    # hyperparameters sent by the client are passed as command-line arguments to the script.
    # to simplify the demo we don't use all sklearn RandomForest hyperparameters
    parser.add_argument("--n-estimators", type=int, default=10)
    parser.add_argument("--min-samples-leaf", type=int, default=3)

    # Data, model, and output directories
    parser.add_argument("--model-dir", type=str, default=os.environ.get("SM_MODEL_DIR"))
    parser.add_argument("--train", type=str, default=os.environ.get("SM_CHANNEL_TRAIN"))
    parser.add_argument("--test", type=str, default=os.environ.get("SM_CHANNEL_TEST"))
    parser.add_argument("--train-file", type=str, default="boston_train.csv")
    parser.add_argument("--test-file", type=str, default="boston_test.csv")
    parser.add_argument(
        "--features", type=str
    )  # in this script we ask user to explicitly name features
    parser.add_argument(
        "--target", type=str
    )  # in this script we ask user to explicitly name the target

    args, _ = parser.parse_known_args()

    print("reading data")
    train_df = pd.read_csv(os.path.join(args.train, args.train_file))
    test_df = pd.read_csv(os.path.join(args.test, args.test_file))

    print("building training and testing datasets")
    X_train = train_df[args.features.split()]
    X_test = test_df[args.features.split()]
    y_train = train_df[args.target]
    y_test = test_df[args.target]

    # train
    print("training model")
    model = RandomForestRegressor(
        n_estimators=args.n_estimators, min_samples_leaf=args.min_samples_leaf, n_jobs=-1
    )

    model.fit(X_train, y_train)

    # print abs error
    print("validating model")
    abs_err = np.abs(model.predict(X_test) - y_test)

    # print couple perf metrics
    for q in [10, 50, 90]:
        print("AE-at-" + str(q) + "th-percentile: " + str(np.percentile(a=abs_err, q=q)))

    # persist model
    path = os.path.join(args.model_dir, "model.joblib")
    joblib.dump(model, path)
    print("model persisted at " + path)
    print(args.min_samples_leaf)



Overwriting script.py


In [50]:
! python script.py --n-estimators 100 \
                   --min-samples-leaf 2 \
                   --model-dir ./ \
                   --train ./ \
                   --test ./ \
                   --features 'CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT' \
                   --target target

extracting arguments
reading data
building training and testing datasets
training model
validating model
AE-at-10th-percentile: 0.2662371428571497
AE-at-50th-percentile: 1.5540277777777725
AE-at-90th-percentile: 4.170540952380944
model persisted at ./model.joblib
2


In [51]:
# We use the Estimator from the SageMaker Python SDK
from sagemaker.sklearn.estimator import SKLearn

FRAMEWORK_VERSION = "0.23-1"

sklearn_estimator = SKLearn(
    entry_point="script.py",
    role=get_execution_role(),
    instance_count=1,
    instance_type="ml.m4.xlarge",#"ml.c5.xlarge",#"ml.m4.xlarge",
    framework_version=FRAMEWORK_VERSION,
    base_job_name="rf-scikit",
    metric_definitions=[{"Name": "median-AE", "Regex": "AE-at-50th-percentile: ([0-9.]+).*$"}],
    hyperparameters={
        "n-estimators": 100,
        "min-samples-leaf": 3,
        "features": "CRIM ZN INDUS CHAS NOX RM AGE DIS RAD TAX PTRATIO B LSTAT",
        "target": "target",
    },
)

import time
tic = time.clock()

# launch training job, with asynchronous call
sklearn_estimator.fit({"train": trainpath, "test": testpath}, wait=False)

toc = time.clock()
print(toc - tic)

0.0799999999999983


In [52]:
sklearn_estimator.latest_training_job.wait(logs="None")
artifact = sm_boto3.describe_training_job(
    TrainingJobName=sklearn_estimator.latest_training_job.name
)["ModelArtifacts"]["S3ModelArtifacts"]

print("Model artifact persisted at " + artifact)

from sagemaker.sklearn.model import SKLearnModel

model = SKLearnModel(
    model_data=artifact,
    role=get_execution_role(),
    entry_point="script.py",
    framework_version=FRAMEWORK_VERSION,
)

predictor = model.deploy(instance_type="ml.c5.large", initial_instance_count=1)




2021-08-24 17:28:57 Starting - Launching requested ML instances.............
2021-08-24 17:30:06 Starting - Preparing the instances for training.................
2021-08-24 17:31:39 Downloading - Downloading input data..
2021-08-24 17:31:54 Training - Downloading the training image.......
2021-08-24 17:32:33 Training - Training image download completed. Training in progress.
2021-08-24 17:32:39 Uploading - Uploading generated training model.
2021-08-24 17:32:47 Completed - Training job completed
Model artifact persisted at s3://sagemaker-us-east-2-296976521096/rf-scikit-2021-08-24-17-28-55-658/output/model.tar.gz
-------------!

In [53]:

n=0
while n<400:
    print(predictor.predict(testX[data.feature_names]))
    n=n+1

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

...

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

[22.69456427 31.21150848 16.87177468 23.7313373  16.87537965 20.93168682
 19.65728799 15.61796595 21.31506209 21.26590754 20.54602367 19.84105278
  8.29822092 21.32990979 19.956743   24.89249708 18.74894055  8.39131295
 44.7281776  15.61190675 24.5023246  23.7948877  14.92396102 23.47977341
 14.57812565 15.23712778 21.56202839 13.99689524 19.13722817 21.08352991
 19.99411263 23.34135541 27.498639   20.14821955 14.9899031  15.74168019
 34.04818265 19.30797778 21.54986932 24.17545188 19.80944755 29.32657565
 44.83337998 19.21722673 22.64661375 13.99425263 15.34171658 24.27799037
 18.74115833 29.11963151 20.9827953  33.90315483 16.97503786 26.27018135
 44.90756374 22.09932662 15.56189192 31.92270844 21.73540884 20.85869181
 25.3593425  34.18939481 30.68533198 18.62262792 27.80028005 16.75548885
 13.33343597 23.33800444 29.01784971 15.23643063 20.47945927 26.85524286
 10.16477551 22.17204206 21.79891014  7.60388319 20.14104722 45.66946649
 10.81814307 13.51945397 21.5722369  10.83278828 20

In [44]:
#sm_boto3.delete_endpoint(EndpointName=predictor.endpoint_name)